# This notebook is to prototype vehicle make prediction

## Establish connection with EVA server

In [ ]:
import sys
sys.path.insert(0,'..')
from src.server.db_api import connect

In [ ]:
import nest_asyncio
nest_asyncio.apply()
connection = connect(host = '0.0.0.0', port = 5432) # hostname, port of the server where EVADB is running

In [ ]:
cursor = connection.cursor()

### Upload the survellience video to analyse

In [ ]:
cursor.execute('UPLOAD INFILE "../data/bdd_test/bdd_1.mp4" PATH "video.mp4";')
response = cursor.fetch_all()
print(response)

## Visualize video

In [ ]:
from ipywidgets import Video
Video.from_file("../data/bdd_test/bdd_1.mp4", embed=True)

## Load video into EVA

In [ ]:
cursor.execute('LOAD DATA INFILE "video.mp4" INTO MyVideo;')
response = cursor.fetch_all()
print(response)

## Definte a new UDF called VehicleMakePredictor if it does not already exist

In [ ]:
cursor.execute("""CREATE UDF IF NOT EXISTS VehicleMakePredictor
      INPUT  (frame NDARRAY UINT8(3, ANYDIM, ANYDIM))
      OUTPUT (labels NDARRAY STR(ANYDIM), bboxes NDARRAY FLOAT32(ANYDIM, 4),
                scores NDARRAY FLOAT32(ANYDIM))
      TYPE  Classification
      IMPL  'src/udfs/vehicle_make_predictor.py';
      """)
response = cursor.fetch_all()
print(response)

## Run vehicle predictor on video

In [ ]:
cursor.execute("""SELECT id, Unnest(VehicleMakePredictor(data)) FROM MyVideo""")
response = cursor.fetch_all()

In [ ]:
response.batch.frames

### Visualize output of vehicle make detector on the video

In [ ]:
import cv2
def annotate_video(detections, input_video_path, output_video_path):
    color=(0,255,0)
    thickness=3

    vcap = cv2.VideoCapture(input_path)
    width = int(vcap.get(3))
    height = int(vcap.get(4))
    fps = vcap.get(5)

    frame_id = 0
    # Capture frame-by-frame
    ret, frame = vcap.read()  # ret = 1 if the video is captured; frame is the image

    while ret:
        df = detections
        df = df[['boxes', 'labels', 'makes']][df.id == frame_id]
        if df.size:
            for bbox, label, make in df.values:
                # Though the model is trained on the actual car version itself, lets just print out the maker for now
                maker = make[0][0].split(" ")[0]
                x1, y1 = bbox[0]
                x2, y2 = bbox[1]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                img=cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness) 
                cv2.putText(img, maker, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, thickness-1) # object label
            
            cv2.imshow('Frame', img)
            # Press Q on keyboard to  exit
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
                            
        frame_id+=1
        ret, frame = vcap.read()

    vcap.release()
    cv2.destroyAllWindows()

In [ ]:
from ipywidgets import Video
input_path = '../data/bdd_test/bdd_1.mp4'
annotate_video(response.batch.frames, input_path, output_path)